In [1]:
# Prerequisites 

from pathlib import Path
import pandas as pd
from ordered_set import OrderedSet
import glob
import seaborn as sns
import matplotlib.pyplot as plt
import yaml
import json
from yaml.loader import SafeLoader
import cobra
from cobra.io import load_model
from cobra import Model, Reaction, Metabolite
from pathlib import Path
from cobra.io import load_json_model, save_json_model, load_matlab_model, save_matlab_model, read_sbml_model, write_sbml_model, load_model, load_yaml_model, save_yaml_model
import logging
from cobra.sampling import sample
from cobra.medium import minimal_medium
import ast
import sys
import numpy as np

# load model

In [2]:
eg_path = Path("/Users/philine/Downloads/yeast-GEM-8.6.2/model")
yml_path = eg_path / "yeast-GEM.yml"
model = load_yaml_model(str(yml_path.resolve()))
model

Restricted license - for non-production use only - expires 2024-10-28


Name,None
Memory address,7f9e4fcd71d0
Number of metabolites,2744
Number of reactions,4063
Number of genes,1160
Number of groups,0
Objective expression,1.0*r_2111 - 1.0*r_2111_reverse_58b69
Compartments,"cell envelope, cytoplasm, extracellular, mitochondrion, nucleus, peroxisome, endoplasmic reticulum, Golgi, lipid particle, vacuole, endoplasmic reticulum membrane, vacuolar membrane, Golgi membrane, mitochondrial membrane"


# load csvs

In [3]:
# read csv of growing pathways
growth_csv = pd.read_csv('final.csv')
growth_csv['INTERMEDIATE NAMES'] = [ast.literal_eval(i) for i in growth_csv['INTERMEDIATE NAMES']]
growth_csv['INTERMEDIATES KEGG'] = [ast.literal_eval(i) for i in growth_csv['INTERMEDIATES KEGG']]
growth_csv

,LENGTH,INTERMEDIATES KEGG,INTERMEDIATE NAMES,solution
0,6,"[C06793, C20609, C00288, C00222, C10020, C0460...","[Vinyl chloride, 2-Chloroacrylate, H2CO3, 3-Ox...",9.379374e+00
1,6,"[C06793, C06614, C06613, C01471, C05608, C0656...","[Vinyl chloride, trans-3-Chloroacrylic acid, t...",1.186644e-17
2,6,"[C06793, C06614, C00288, C00222, C10020, C0460...","[Vinyl chloride, trans-3-Chloroacrylic acid, H...",9.379374e+00
3,6,"[C06793, C20609, C00288, C00222, C10020, C0460...","[Vinyl chloride, 2-Chloroacrylate, H2CO3, 3-Ox...",9.379374e+00
4,6,"[C06793, C06614, C00288, C00222, C10020, C0460...","[Vinyl chloride, trans-3-Chloroacrylic acid, H...",9.379374e+00
5,6,"[C06793, C06614, C06613, C01471, C00903, C1640...","[Vinyl chloride, trans-3-Chloroacrylic acid, t...",2.138682e-17
6,6,"[C06793, C06753, C00007, C14448, C00546, C1972...","[Vinyl chloride, 2-Chloroethanol, Oxygen, Glyo...",6.126559e+00
7,6,"[C06793, C20303, C00007, C00067, C00546, C1972...","[Vinyl chloride, Chloroethylene oxide, Oxygen,...",6.126559e+00
8,6,"[C06793, C20303, C00007, C00067, C00546, C1228...","[Vinyl chloride, Chloroethylene oxide, Oxygen,...",6.126559e+00
9,6,"[C06793, C20303, C00007, C00048, C00546, C1972...","[Vinyl chloride, Chloroethylene oxide, Oxygen,...",6.322964e+00


In [4]:
# read csv of pathway generating highest yield 
max_pathway = pd.read_csv('max_pathway.csv')
max_pathway['INTERMEDIATE NAMES'] = [ast.literal_eval(i) for i in max_pathway['INTERMEDIATE NAMES']]
max_pathway['INTERMEDIATES KEGG'] = [ast.literal_eval(i) for i in max_pathway['INTERMEDIATES KEGG']]
max_pathway

,LENGTH,INTERMEDIATES KEGG,INTERMEDIATE NAMES,solution
0,6,"[C06793, C20609, C00288, C00222, C10020, C0460...","[Vinyl chloride, 2-Chloroacrylate, H2CO3, 3-Ox...",9.379374


## select pathway to utilise

In [5]:
df = max_pathway

# prep model

In [6]:
# preapre model and apply to yeast model
def prep_model(model):
    
    model.solver = 'glpk'
    model.objective = 'r_4041'
    
    medium = model.medium
    medium['r_1714'] = 0.0
    model.medium = medium
    
    return model

model = prep_model(model)

In [7]:
#get csv containing s id and kegg id
sid_kegg = pd.read_csv('map_sid_kegg.csv')


# create dictionary of sid_kegg data frame
sk_dict = dict(zip(sid_kegg['S ID'], sid_kegg['KEGG ID']))
sk_dict

{'s_0002': 'C00965',
 's_0007': 'C11514',
 's_0009': 'C04411',
 's_0010': 'C04236',
 's_0013': nan,
 's_0015': 'C03406',
 's_0017': 'C03492',
 's_0018': 'C01143',
 's_0019': 'C01107',
 's_0020': 'C00810',
 's_0021': 'C00318',
 's_0025': 'C00256',
 's_0028': 'C00418',
 's_0030': 'C00522',
 's_0031': 'C00864',
 's_0033': 'C03451',
 's_0035': 'C03044',
 's_0037': 'C01054',
 's_0056': 'C00671',
 's_0061': 'C00337',
 's_0062': 'C00424',
 's_0063': 'C00186',
 's_0066': 'C00149',
 's_0075': 'C00236',
 's_0076': 'C01302',
 's_0077': 'C04896',
 's_0078': 'C02741',
 's_0079': 'C01225',
 's_0086': 'C03506',
 's_0087': 'C02918',
 's_0089': 'C01194',
 's_0116': 'C04281',
 's_0118': 'C04322',
 's_0120': 'C00234',
 's_0122': 'C05108',
 's_0126': 'C01177',
 's_0131': 'C02353',
 's_0132': 'C00559',
 's_0134': 'C00330',
 's_0136': 'C05512',
 's_0138': 'C00526',
 's_0140': 'C01159',
 's_0141': 'C01304',
 's_0142': 'C18910',
 's_0143': 'C04441',
 's_0144': 'C04692',
 's_0147': 'C04409',
 's_0149': 'C00966

# get plastics

In [8]:
# get all plastics

def get_plastics(df):
    
    csv_pathways = [i for i in df['INTERMEDIATES KEGG']]

    get_plastic = set()

    for i in csv_pathways:
        if i[0] not in get_plastic:
            get_plastic.add(i[0])

    plastics = list(get_plastic)
    return plastics

In [9]:
# apply to dataframe
plastics = get_plastics(df)
plastics

['C06793']

In [10]:
# create plastic metabolites

## Propene

C11505_e = Metabolite(
    'C11505_e',
    formula = 'C3H6',
    name = 'Propene',
    compartment = 'e')

C11505_c = Metabolite(
    'C11505_c',
    formula = 'C3H6',
    name = 'Propene',
    compartment = 'c')


## Vinyl Chloride

C06793_e = Metabolite(
    'C06793_e',
    formula = 'C2H3Cl',
    name = 'Vinyl Chloride',
    compartment = 'e')

C06793_c = Metabolite(
    'C06793_c',
    formula = 'C2H3Cl',
    name = 'Vinyl Chloride',
    compartment = 'c')



## Ethylene

C06547_e = Metabolite(
    'C06547_e',
    formula = 'C2H4',
    name = 'Ethylene',
    compartment = 'e')

C06547_c = Metabolite(
    'C06547_c',
    formula = 'C2H4',
    name = 'Ethylene',
    compartment = 'c')



## Styrene

C07083_e = Metabolite(
    'C07083_e',
    formula = 'C8H8',
    name = 'Styrene',
    compartment = 'e')

C07083_c = Metabolite(
    'C07083_c',
    formula = 'C8H8',
    name = 'Styrene',
    compartment = 'c')


## Terephthalate 

C06337_e = Metabolite(
    'C06337_e',
    formula = 'C8H6O4',
    name = 'Terephthalate',
    compartment = 'e')

C06337_c = Metabolite(
    'C06337_c',
    formula = 'C8H6O4',
    name = 'Terephthalate',
    compartment = 'c')

In [11]:
# add plastic dictionary and add metabolites
plastic_dict = {}

plastic_dict['C11505_c'] = C11505_c
plastic_dict['C06793_c'] = C06793_c
plastic_dict['C06547_c'] = C06547_c
plastic_dict['C07083_c'] = C07083_c
plastic_dict['C06337_c'] = C06337_c

plastic_dict['C11505_e'] = C11505_e
plastic_dict['C06793_e'] = C06793_e
plastic_dict['C06547_e'] = C06547_e
plastic_dict['C07083_e'] = C07083_e
plastic_dict['C06337_e'] = C06337_e

In [12]:
# add plastic metabolites to dictionary

sk_dict['C11505_c'] = C11505_c
sk_dict['C06793_c'] = C06793_c
sk_dict['C06547_c'] = C06547_c
sk_dict['C07083_c'] = C07083_c
sk_dict['C06337_c'] = C06337_c

sk_dict['C11505_e'] = C11505_e
sk_dict['C06793_e'] = C06793_e
sk_dict['C06547_e'] = C06547_e
sk_dict['C07083_e'] = C07083_e
sk_dict['C06337_e'] = C06337_e

# run fva

In [13]:
# create algorithm calculating flux variability analysis
def fva(df, model, plastics):
    
    # get pathways from data frame
    pathways = [i for i in df['INTERMEDIATES KEGG']]
    
    
    
    # change name of plastics so that it matches version in dict
    for i in pathways:
        for e, compound in enumerate(i):
            if compound in plastics:
                i[e] = compound + '_c'
                
    
    
    # get list of cytosolic plastics (for later analysis)
    plastics_c = plastics.copy()

    for e, compound in enumerate(plastics_c):
        plastics_c[e] = compound + '_c'
     
    
            
    # get external plastics        
    e_plastics = plastics.copy()
    
    for i, plastic in enumerate(plastics):
        e_plastics[i] = plastic + '_e'
   
     
    
    # get list of products    
    products = [i[-1] for i in pathways]
    products = list(set(products))
    
    
    
    # get external products
    e_products = products.copy()
    
    for i, prod in enumerate(products):
        e_products[i] = prod + '_e'

    
    
    # get name of compounds that are not present in model
    get_compound = set()

    for pathway in pathways:
        for compound in pathway:
            if compound not in plastic_dict and compound not in sk_dict.values() and compound not in get_compound:
                get_compound.add(compound)

    get_compound = list(get_compound)
    
    

    # create names for missing metabolites
    compound_name = []

    for i, compound in enumerate(get_compound):
        comp = f"Metabolite_{i}"
        compound_name.append(comp)

        
    
    # take list of missing compounds, create metabolites and add to dictionaries
    metabolites = []
    met_dict = {}

    for compound in get_compound:
        new_met = Metabolite(compound, compartment='c')
        metabolites.append(new_met)
        sk_dict[compound] = new_met
        met_dict[compound] = new_met
        
    
    
    # add external plastics to list of pathways
    ext = []
    
    for pathway in pathways:
        if pathway[0] in plastics_c:
            index = plastics_c.index(pathway[0])
            ext.append(e_plastics[index])
        
        
    
    # add external products (weren't in list before) and add to model
    ext_p = []

    for i in e_products:
        new_prod = Metabolite(i, compartment='e')
        ext_p.append(new_prod)
        sk_dict[i] = new_prod

    model.add_metabolites(ext_p)
    
    
    # add external plastics to list of pathways
    ext2 = []
    
    for pathway in pathways:
        if pathway[-1] in products:
            index = products.index(pathway[-1])
            ext2.append(e_products[index])

    
            
    # add external lists to pathways     
    for i, item in enumerate(ext):
        pathways[i].insert(0, item)
    
    for j, item in enumerate(ext2):
        pathways[j].append(item)

        
        
    # transform coompounds in pathways list to metabolites
    for i in pathways:
        for j, item in enumerate(i):
            
            found = False
            

            if item in sk_dict:
                i[j] = sk_dict[item]
                found = True

            if not found and item in met_dict:
                i[j] = sk_dict[item]
                found = True

            if not found and item in sk_dict.values():
                temp = [key for key, value in sk_dict.items() if value == item]
                if temp:
                    temp2 = temp[0]
                    met = model.metabolites.get_by_id(temp2)
                    i[j] = met
                    found = True

            if not found:
                i[j] = item


    
    # separate list of pathways into twos (metabolite_1 --> metabolite 2)
                                        # (metabolite_2 --> metabolite_3)
                                        #  etc. 
    separated_pathways = []

    for i in pathways:
        pathway_pairs = []
        for j in range(len(i)-1):
            pair = [i[j], i[j+1]]
            pathway_pairs.append(pair)
        separated_pathways.append(pathway_pairs)
        
    
    
    # get number of reactions in pathway
    react_num = 0
    for i in separated_pathways:
        react_num += len(i)
        
        
     
    # create empty reactions using number above
    reactions = []

    for i in range(react_num):
        reaction = f"Reaction_{i}"

        new_reaction = Reaction(reaction)

        reactions.append(new_reaction)
        
        
        
    # create output list    
    grows = []
   

    
    # add metabolites to reactions and reaction to model
    for m, e in enumerate(separated_pathways):
        
        reactions_added = []
        
        for i, item in enumerate(e):
            
            # Add reaction to the model
            model.add_reactions([reactions[i]])
            
            # Add metabolites to reaction
            reactions[i].add_metabolites({
                    item[0]: -1.0,
                    item[1]: 1.0
                    })


            # for first and last reactions, create exchange reactions
            if i == 0 or i == len(e)-1:
                reactions[i].lower_bound = -10.0
                reactions[i].upper_bound = 1000.0
                
                
            # for all others, create normal (oneway) reactions
            else:
                reactions[i].lower_bound = 0.0
                reactions[i].upper_bound = 1000.0
 

            reactions_added.append(reactions[i])
            
            
            
        # run FBA on model    
        solution = model.optimize()
        
        fva = []
        
        # if cell can grow, store results
        if solution.status == 'optimal' and solution.objective_value > 0.000:

            grows.append(solution)
            
            
            fva_result = cobra.flux_analysis.flux_variability_analysis(model, model.reactions[:], fraction_of_optimum=0.9)
            pd.DataFrame.from_dict(fva_result).T
            
            return (solution, fva_result)
            
            #fva.append(fva_result)
            
            # remove all reactions and metabolites added within the loop
            model.remove_reactions(reactions_added)

            reactions = [reaction for reaction in reactions if reaction not in reactions_added]

            del reactions_added[:]
            
            

            

        else:
            
            # remove all reactions and metabolites added within the loop
            model.remove_reactions(reactions_added)

            reactions = [reaction for reaction in reactions if reaction not in reactions_added]

            del reactions_added[:]
            
    
    
    return (solution, fva_result)

# results

In [14]:
# call algorithm using highest pathway
a = fva(df, model, plastics)
a

(<Solution 9.379 at 0x7f9e3c294f10>,
                minimum      maximum
 r_0001        0.000000    30.645530
 r_0002        0.000000    30.645530
 r_0003     -340.313838     0.000000
 r_0004        0.000000    27.819494
 r_0005        6.318542     7.020602
 ...                ...          ...
 Reaction_3  561.465198  1000.000000
 Reaction_4    0.000000     0.000000
 Reaction_5    0.000000     0.000000
 Reaction_6    0.000000     0.000000
 Reaction_7    0.000000     0.000000
 
 [4071 rows x 2 columns])

In [15]:
# get dataframe from fva results

In [16]:
fva_result = a[1]
fva_result

,minimum,maximum
r_0001,0.000000,30.645530
r_0002,0.000000,30.645530
r_0003,-340.313838,0.000000
r_0004,0.000000,27.819494
r_0005,6.318542,7.020602
...,...,...
Reaction_3,561.465198,1000.000000
Reaction_4,0.000000,0.000000
Reaction_5,0.000000,0.000000
Reaction_6,0.000000,0.000000


In [17]:
# top reactions form fba
top_20 = ['r_2100',
 'r_1667',
 'r_0438',
 'r_1277',
 'r_1110',
 'r_2096',
 'r_1245',
 'r_0226',
 'r_1763',
 'r_1632',
 'r_0165',
 'r_4575',
 'r_1761',
 'r_1762',
 'r_1696',
 'r_0439',
 'r_0658',
 'r_4574',
 'r_1672',
 'r_1697']

In [18]:
# generate new df wiht top 20 flux range
top_fva = fva_result.loc[top_20]
top_fva

,minimum,maximum
r_2100,-1000.000000,-338.135323
r_1667,-426.076688,1000.000000
r_0438,878.498633,1000.000000
r_1277,338.135323,1000.000000
r_1110,753.888060,902.271452
r_2096,-1000.000000,-314.474198
r_1245,518.022302,940.861777
r_0226,493.788199,821.027644
r_1763,-1000.000000,173.478743
r_1632,-1000.000000,-192.954838


# load model data

In [19]:
# loading yeast metabolic model

model_path = '/Users/philine/Downloads/yeast-GEM-8.6.2/model/yeast-GEM.yml'

with open(model_path) as file:
    data = yaml.load(file, Loader=yaml.FullLoader)

In [20]:
# isolate reactions data
reactions = data[2]
reactions

('reactions',
 [[('id', 'r_0001'),
   ('name', '(R)-lactate:ferricytochrome-c 2-oxidoreductase'),
   ('metabolites',
    [('s_0025', -1), ('s_0709', -2), ('s_0710', 2), ('s_1399', 1)]),
   ('lower_bound', 0),
   ('upper_bound', 1000),
   ('gene_reaction_rule',
    '(YDL174C and YEL039C) or (YDL174C and YJR048W) or (YEL039C and YEL071W) or (YEL071W and YJR048W)'),
   ('eccodes', ['1.1.2.4', '1.1.99.-']),
   ('subsystem', ['Pyruvate metabolism']),
   ('annotation',
    [('bigg.reaction', 'D_LACDcm'),
     ('kegg.pathway', ['sce00620', 'sce00920']),
     ('kegg.reaction', 'R00197'),
     ('metanetx.reaction', 'MNXR138960'),
     ('sbo', 'SBO:0000176')]),
   ('confidence_score', 2)],
  [('id', 'r_0002'),
   ('name', '(R)-lactate:ferricytochrome-c 2-oxidoreductase'),
   ('metabolites',
    [('s_0027', -1), ('s_0709', -2), ('s_0710', 2), ('s_1401', 1)]),
   ('lower_bound', 0),
   ('upper_bound', 1000),
   ('gene_reaction_rule', '(YDL178W and YEL039C) or (YDL178W and YJR048W)'),
   ('eccodes'

In [21]:
# get all names of reactions
names = []
for i in reactions[1][:20]:
    for j in i:
        if j[0] == 'name':
            names.append(j[1])

names

['(R)-lactate:ferricytochrome-c 2-oxidoreductase',
 '(R)-lactate:ferricytochrome-c 2-oxidoreductase',
 '(R,R)-butanediol dehydrogenase',
 '(S)-lactate:ferricytochrome-c 2-oxidoreductase',
 '1,3-beta-glucan synthase',
 '1,6-beta-glucan synthase',
 '1-(5-phosphoribosyl)-5-[(5-phosphoribosylamino)methylideneamino)imidazole-4-carboxamide isomerase',
 '1-pyrroline-5-carboxylate dehydrogenase',
 '2,3-diketo-5-methylthio-1-phosphopentane degradation reaction',
 "2,5-diamino-6-ribitylamino-4(3H)-pyrimidinone 5'-phosphate deaminase",
 "2,5-diamino-6-ribosylamino-4(3H)-pyrimidinone 5'-phosphate reductase (NADPH)",
 '2-aceto-2-hydroxybutanoate synthase',
 '2-amino-4-hydroxy-6-hydroxymethyldihydropteridine diphosphokinase',
 '2-aminoadipate transaminase',
 '2-dehydropantoate 2-reductase',
 '2-deoxy-D-arabino-heptulosonate 7-phosphate synthetase',
 '2-hexaprenyl-6-methoxy-1,4-benzoquinone methyltransferase',
 '2-hexaprenyl-6-methoxyphenol monooxygenase',
 '2-isopropylmalate hydratase',
 '2-isopropy

In [22]:
# get names of top 20 fluxes
top_names = []

for i in reactions[1]:
    #print(i[0][1])
    #for j in i:
     #   print(j[0])
    if i[0][1] in top_20:
        #print(i[0][1])
        for j in i:
            #print(j[0])
            if j[0] == 'name':

                top_names.append(j[1])

top_names

['mitochondrial alcohol dehydrogenase',
 'ATP synthase',
 'ferrocytochrome-c:oxygen oxidoreductase',
 'ubiquinol:ferricytochrome c reductase',
 'isocitrate dehydrogenase (NAD+)',
 'ADP/ATP transporter',
 'phosphate transport',
 'water diffusion',
 'acetaldehyde transport',
 'bicarbonate formation',
 'carbon dioxide exchange',
 'CO2 transport',
 'CO2 transport',
 'ethanol exchange',
 'ethanol transport',
 'ethanol transport, mitochondrial',
 'water diffusion',
 'water exchange',
 '3-Oxopropanoate:NADP+ oxidoreductase (decarboxylating, CoA-acetylating)',
 '3-oxopropanoate carboxy-lyase']

In [23]:
# add reaction name to df
top_fva['reaction name'] = ''
top_fva['reaction name'] = [i for i in top_names]
top_fva

,minimum,maximum,reaction name
r_2100,-1000.000000,-338.135323,mitochondrial alcohol dehydrogenase
r_1667,-426.076688,1000.000000,ATP synthase
r_0438,878.498633,1000.000000,ferrocytochrome-c:oxygen oxidoreductase
r_1277,338.135323,1000.000000,ubiquinol:ferricytochrome c reductase
r_1110,753.888060,902.271452,isocitrate dehydrogenase (NAD+)
r_2096,-1000.000000,-314.474198,ADP/ATP transporter
r_1245,518.022302,940.861777,phosphate transport
r_0226,493.788199,821.027644,water diffusion
r_1763,-1000.000000,173.478743,acetaldehyde transport
r_1632,-1000.000000,-192.954838,bicarbonate formation
